In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
url = 'http://transparentcalifornia.com/salaries/2016/university-of-california/'

payload = requests.get(url) 
soup = BeautifulSoup(payload.text)

/home/lachlan/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
# get the table where salaries are
salaryTable = soup.tbody

## Processing One Row of data

In [4]:
# find employee
employee = salaryTable('tr')[0]
print(employee)

<tr>
<td>
<a href="/salaries/2016/university-of-california/james-lawrence-mora/">James Lawrence Mora</a>
</td>
<td>
<a href="/salaries/search/?q=INTERCOL%20ATH%20HEAD%20COACH%20EX&amp;y=2016">INTERCOL ATH HEAD COACH EX</a><br/>
<small class="muted"><a href="/salaries/2016/university-of-california/">University of California, 2016</a></small>
</td>
<td>$300,000.00</td>
<td>$0.00</td>
<td>$3,277,299.00</td>
<td>$3,577,299.00</td>
<td>Not provided</td>
<td>$3,577,299.00</td>
</tr>


In [5]:
def get_employee_name(employee_data):
    employee_name = employee_data.findNext('a', href=True)
    employee_name_text = employee_name.text
    
    return employee_name_text

In [6]:
get_employee_name(employee)

'James Lawrence Mora'

In [7]:
def get_employee_job(employee_data):
    employee_job = employee_data.findNext('a', href=True).findNext('a', href=True)
    employee_job_text = employee_job.text.lower()
    
    return employee_job_text

In [8]:
get_employee_job(employee)

'intercol ath head coach ex'

In [9]:
def money_to_int(moneystring):
    '''
    A helper function to strip out dollar signs ($) and commas leaving any
    dollar value as a integer
    '''
    try:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return float(moneystring)
    except:
        pass
    return moneystring

def get_total_pay(employee_data):
    total_income = employee_data('td')[-1].text
    return money_to_int(total_income)

In [10]:
get_total_pay(employee)

3577299.0

In [11]:
# Packing it into a row of data

In [12]:
def process_employee(employee_data):
    import pandas as pd
    
    name      = get_employee_name(employee_data)
    job_title = get_employee_job(employee_data)
    salary   = get_total_pay(employee_data)
    
    # make a row of data
    headers = ['name','job', 'salary']
    
    df_employee = pd.DataFrame([[name, job_title, salary]],
                                    columns=headers)

    # return a line of data to the object assigned
    return df_employee

In [13]:
process_employee(employee)

,name,job,salary
0,James Lawrence Mora,intercol ath head coach ex,3577299.0


## Getting all data from the table

In [14]:
import pandas as pd

# empty data frame
df_staff = pd.DataFrame()

# for every employee:
for iEmployee in salaryTable.findAll('tr'):
    
    df_employee = process_employee(iEmployee)
    df_staff = df_staff.append(df_employee, ignore_index=True)

In [15]:
df_staff

,name,job,salary
0,James Lawrence Mora,intercol ath head coach ex,3577299.0
1,Cuonzo Martin,head coach 5,2933098.0
2,Daniel Dykes,head coach 5,2891233.0
3,Stephen Todd Alford,intercol ath head coach ex,2721405.0
4,Gordon Alan Cohen,hs clin instr-hcomp,2720796.0
5,Ronald W Busuttil,prof-hcomp,2482207.0
6,Vadiyala Mohan Reddy,act prof-hcomp,1943521.0
7,Timothy H Mccalmont,prof of clin-hcomp,1926941.0
8,Eric Esrailian,hs assoc clin prof-hcomp,1837625.0
9,Philip E Leboit,prof of clin-hcomp,1779457.0
